In [19]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import cv2
import math
import csv
from PIL import Image
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime

In [20]:
ratio = 4
channels = 3
HR_size = np.array([480,640])#
LR_size = HR_size // ratio
H_h = HR_size[0]
H_w = HR_size[1]
L_h = LR_size[0]
L_w = LR_size[1]
print(HR_size[0])
print(LR_size)

480
[120 160]


In [21]:
class DataLoader():
    def __init__(self):
        pass
#         self.ratio = ratio
#         self.HR_size = HR_size
#         self.LR_size = HR_size / ratio
#         self.H_h = HR_size[0]
#         self.H_w = HR_size[1]
#         self.L_h = LR_size[0]
#         self.L_h = LR_size[1]
        
    
    
    def load_data(self, batch_size = 1):
        files = glob.glob("../../images/train/*.png", recursive=True)
        batch_images = random.sample(files, batch_size)
    
        HR_images = []
        LR_images = []
    
        for img_path in batch_images:
            img = Image.open(img_path)

            HR_image = img.resize((H_w, H_h))  #(64, 64)
            LR_image = img.resize((L_w, L_h))
            HR_image = np.array(HR_image)
            #img_hr = (img_hr - 127.5) / 127.5
            LR_image = np.array(LR_image)
            #img_lr = (img_lr - 127.5) / 127.5

#             if not is_testing and np.random.random() < 0.5:
#                 img_hr = np.fliplr(img_hr)
#                 img_lr = np.fliplr(img_lr)

            HR_images.append(HR_image)
            LR_images.append(LR_image)
        
        HR_images = np.array(HR_images) / 127.5 - 1
        LR_images = np.array(LR_images) / 127.5 - 1
        
        return HR_images, LR_images

#------------------------------------------------------------------#
  
#------------------------------------------------------------------#

class predDataLoader():          
    def load_data(self, batch_size, counter):
        random.seed(counter)
        np.random.seed(counter)

        files = glob.glob("../../images/test/*.png", recursive=True)
        batch_images = random.sample(files, batch_size)

        imgs_or = []
        for img_path in batch_images:
            img = Image.open(img_path)

            img_or = np.array(img)
            imgs_or.append(img_or)

        imgs_or = np.array(imgs_or) / 127.5 - 1.

        return imgs_or
    
    
#------------------------------------------------------------------#
  
#------------------------------------------------------------------#

class SRGAN():
    def __init__(self):
        
        # Input shape
#         self.channels = 3
#         self.lr_height = 288                 # Low resolution height
#         self.lr_width = 384                  # Low resolution width
#         self.lr_shape = (self.lr_height, self.lr_width, self.channels)
#         resLevel = 2 #
#         self.hr_height = self.lr_height*resLevel  # High resolution height
#         self.hr_width = self.lr_width*resLevel     # High resolution width
#         self.hr_shape = (self.hr_height, self.hr_width, self.channels)

        #残差ブロックの数
        # Number of residual blocks in the generator
        self.n_residual_blocks = 16 #

        optimizer = Adam(0.0002, 0.5)

        #img_hrの特徴量をVGG19で算出する
        # We use a pre-trained VGG19 model to extract image features from the high resolution
        # and the generated high resolution images and minimize the mse between them
        self.VGG = self.build_vgg()
        self.VGG.trainable = False
        self.VGG.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        
        #データはここで読み込まれる
        # Configure data loader
        #self.dataset_name = 'img_align_celeba'
        self.data_loader = DataLoader()
        self.pred_data_loader = predDataLoader()

        #Dのサイズ
        # Calculate output shape of D (PatchGAN)
        patchH = int(H_h / 2**4) #
        patchW = int(H_w / 2**4) #
        self.disc_patch = (patchH, patchW, 1) 

        #DとGのチャンネル設定
        # Number of filters in the first layer of G and D
        self.gf = 60 #gf
        self.df = 60 #df
        #Dビルドとコンパイル
        # Build and compile the discriminator
        self.Discriminator = self.build_discriminator()
        self.Discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #Gのビルド
        # Build the generator
        self.Generator = self.build_generator()

        # High res. and low res. images
        HR_image = Input(shape = (H_h, H_w, channels))
        LR_image = Input(shape = (L_h, L_w, channels))

        #Gで生成されたhrのimg
        # Generate high res. version from low res.
        SR_image = self.Generator(LR_image)

        #hrのimgの特徴量の算出
        # Extract image features of the generated img
        SR_features = self.VGG(SR_image)

        # For the combined model we will only train the generator
        self.Discriminator.trainable = False

        #学習モデルコンパイル
        # Discriminator determines validity of generated high res. images
        validity = self.Discriminator(SR_image)

        self.combined = Model([LR_image, HR_image], [validity, SR_features])
        self.combined.compile(loss=['binary_crossentropy', 'mse'],
                              loss_weights=[1e-3, 1],
                              optimizer=optimizer)
        
    #------------------------------------------------------------------#
    
    def build_vgg(self):
        """
        Builds a pre-trained VGG19 model that outputs image features extracted at the
        third block of the model
        """
        VGG = VGG19(weights="imagenet")
        # Set outputs to outputs of last conv. layer in block 3
        # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
        VGG.outputs = [VGG.layers[9].output]

        img = Input(shape = (H_h, H_w, channels))

        # Extract image features
        img_features = VGG(img)

        return Model(img, img_features)
    
    #------------------------------------------------------------------#
    
    #Generator の実装
    def build_generator(self):

        #残差ブロックの中身
        def residual_block(layer_input, n_filters):
            """Residual block described in paper"""
            d = Conv2D(n_filters, kernel_size=3, strides=1, padding='same')(layer_input)
            d = Activation('relu')(d)
            d = BatchNormalization(momentum=0.8)(d)
            d = Conv2D(n_filters, kernel_size=3, strides=1, padding='same')(d)
            d = BatchNormalization(momentum=0.8)(d)
            d = Add()([d, layer_input])
            return d

        #解像度を2倍にするUpSampling
        def deconv2d(layer_input):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input) #
            u = Conv2D(120, kernel_size=3, strides=1, padding='same')(u) #
            u = Activation('relu')(u)
            return u

        # Low resolution image input
        img_lr = Input(shape = (L_h, L_w, channels))

        # Pre-residual block
        c1 = Conv2D(60, kernel_size=9, strides=1, padding='same')(img_lr) #
        c1 = Activation('relu')(c1)

        # Propogate through residual blocks
        r = residual_block(c1, self.gf)
        for _ in range(self.n_residual_blocks - 1):
            r = residual_block(r, self.gf)

        #去の残差ブロックと組み合わせる
        # Post-residual block
        c2 = Conv2D(60, kernel_size=3, strides=1, padding='same')(r) #
        c2 = BatchNormalization(momentum=0.8)(c2)
        c2 = Add()([c2, c1])

        # Upsampling
        n = ratio
        while(n % 2 == 0):
            c2 = deconv2d(c2)
            n = n // 2

        # Generate high resolution output
        gen_hr = Conv2D(channels, kernel_size=9, strides=1, padding='same', activation='tanh')(c2)

        return Model(img_lr, gen_hr)
    
    
    #------------------------------------------------------------------#
    
    #Discriminator の実装
    def build_discriminator(self):

        
        def d_block(layer_input, filters, strides=1, bn=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=3, strides=strides, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if bn:
                d = BatchNormalization(momentum=0.8)(d)
            return d

        # Input img
        d0 = Input(shape = (H_h, H_w, channels))
        #畳み込み層、チャンネル数を最終的に16倍に
        d1 = d_block(d0, self.df, bn=False)
        d2 = d_block(d1, self.df, strides=2)
        d3 = d_block(d2, self.df*2)
        d4 = d_block(d3, self.df*2, strides=2)
        d5 = d_block(d4, self.df*4)
        d6 = d_block(d5, self.df*4, strides=2)
        d7 = d_block(d6, self.df*8)
        d8 = d_block(d7, self.df*8, strides=2)
        #この時点で画像サイズ1/16
        d9 = Dense(self.df*16)(d8)
        d10 = LeakyReLU(alpha=0.2)(d9)
        validity = Dense(1, activation='sigmoid')(d10)

        return Model(d0, validity)
    
    
    #------------------------------------------------------------------#
    
    
    #学習
    def train(self, epochs, batch_size, sample_interval=100):

        start_time = datetime.datetime.now()
        psnr_file = open('psnr.csv' , 'w+')
        psnr_file.close()

        for epoch in range(epochs):

            # ----------------------
            #  Train Discriminator
            # ----------------------

            # Sample images and their conditioning counterparts
            HR_images, LR_images = self.data_loader.load_data(batch_size)

            # From low res. image generate high res. version
            SR_images = self.Generator.predict(LR_images)

            valid = np.ones((batch_size,) + self.disc_patch)
            fake = np.zeros((batch_size,) + self.disc_patch)

            #Dのloss
            # Train the discriminators (original images = real / generated = Fake)
            d_loss_real = self.Discriminator.train_on_batch(HR_images, valid)
            d_loss_fake = self.Discriminator.train_on_batch(SR_images, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ------------------
            #  Train Generator
            # ------------------

            # Sample images and their conditioning counterparts
           
            HR_images, LR_images = self.data_loader.load_data(batch_size)

            # The generators want the discriminators to label the generated images as real
            valid = np.ones((batch_size,) + self.disc_patch)

            # Extract ground truth image features using pre-trained VGG19 model
            image_features = self.VGG.predict(HR_images)

            # Train the generators
            g_loss = self.combined.train_on_batch([LR_images, HR_images], [valid, image_features])

            elapsed_time = datetime.datetime.now() - start_time
            # Plot the progress
            print ("%d time: %s" % (epoch, elapsed_time))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.Generator.save_weights('srg_weight.h5')
                
                
    #------------------------------------------------------------------#
    
            
    def sample_images(self, epoch):
        def denormalize(input_data):
            input_data = (input_data + 1) * 127.5
            return input_data.astype(np.uint8)
        os.makedirs('../../images/images/%s' , exist_ok=True)
        r, c = 2, 2

        HR_images, LR_images = self.data_loader.load_data(batch_size=2)
        SR_images = self.Generator.predict(LR_images)

        LR_images = denormalize(LR_images)
        SR_images = denormalize(SR_images)
        HR_images = denormalize(HR_images)

        def psnr_calc(img1: np.ndarray, img2: np.ndarray, upscaling=2):
            def convert(img):
                return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            def extract_y(image: np.ndarray) -> np.ndarray:
                if image.ndim == 2:
                    return image
                image = image.astype(np.int32)
                return ((image[:, :, 2] * 65.481 / 255.
                          + image[:, :, 1] * 128.553 / 255.
                          + image[:, :, 0] * 24.966 / 255.) + 16).astype(np.int32)


            def psnr(img1, img2):
                mse = np.mean((img1 - img2) ** 2)
                if mse == 0:
                    return 100
                PIXEL_MAX = 255.0
                return 10 * math.log10(PIXEL_MAX * PIXEL_MAX / mse)
  
            img1_conv=convert(img1)
            img2_conv=convert(img2)
  
            # BGR -> YCrCb
            # 画像はcv2.imreadで読まれている前提 [0, 255]
            y1 = extract_y(img1_conv)
            y2 = extract_y(img2_conv)
            # 周囲のcropping
            # assert y1.shape == y2.shape
            h, w = y1.shape
            cr = upscaling
            cropped_y1 = y1[cr:h - cr, cr:w - cr].astype(np.float64)
            cropped_y2 = y2[cr:h - cr, cr:w - cr].astype(np.float64)

            # psnr
            psnr_val = psnr(cropped_y1, cropped_y2)
            return psnr_val
        
        
        # Save generated images and the high resolution originals
        titles = ['Generated', 'Original']

        cv2.imwrite("../../images/images/hreal_img_0.png",HR_images[0])
        cv2.imwrite("../../images/images/hreal_img_1.png",HR_images[1])
        cv2.imwrite("../../images/images/lreal_img_0.png",LR_images[0])
        cv2.imwrite("../../images/images/lreal_img_1.png",LR_images[1])
        # size=(480,480)

        for i in range(r):
            psnr = psnr_calc(HR_images[i],SR_images[i])
            print(psnr)
            data =[epoch,psnr]
            psnr_file = open('psnr.csv' , 'a')
            writer = csv.writer(psnr_file, lineterminator='\n')  
            writer.writerow(data)
            psnr_file.close()
            cv2.imwrite("../../images/images/{}_{}img_pred.png".format(epoch,i),SR_images[i])
            
            
    #------------------------------------------------------------------#

In [ ]:
if __name__ == '__main__':
    gan = SRGAN()
    gan.train(epochs=500, batch_size=1, sample_interval=50)